<a href="https://colab.research.google.com/github/kimjinseong-ctrl/DScover_Guide_Project_G/blob/main/G_Guide_Project_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

라이브러리 및 폰트 저장

In [21]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, silhouette_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
import warnings
import os
import urllib.request
from matplotlib import font_manager
import matplotlib.pyplot as plt
import seaborn as sns

font_path = 'NanumGothic.ttf'
if not os.path.exists(font_path):
    print("나눔고딕 폰트를 다운로드합니다...")
    url = 'https://github.com/google/fonts/raw/main/ofl/nanumgothic/NanumGothic-Regular.ttf'
    try:
        urllib.request.urlretrieve(url, font_path)
        print("폰트 다운로드 완료.")
    except Exception as e:
        print(f"폰트 다운로드 중 오류가 발생했습니다: {e}")
        font_path = None
if font_path:
    font_prop = font_manager.FontProperties(fname=font_path)
    print(f"'{font_path}' 폰트를 그래프에 적용합니다.")
else:
    font_prop = None
    print("한글 폰트를 설정할 수 없어, 텍스트가 깨질 수 있습니다.")
plt.rcParams['axes.unicode_minus'] = False

'NanumGothic.ttf' 폰트를 그래프에 적용합니다.


데이터 로딩

In [22]:
# --- SMAPE 계산 함수 정의 ---
def smape(actual, forecast):
    epsilon = 1e-10
    return np.mean(2 * np.abs(forecast - actual) / (np.abs(actual) + np.abs(forecast) + epsilon)) * 100

train_df = pd.read_csv('train.csv', encoding='utf-8')
test_df = pd.read_csv('test.csv', encoding='utf-8', dtype={'num_date_time': str})
building_info_df = pd.read_csv('building_info.csv', encoding='utf-8')


건물 그룹핑

In [23]:
cluster_df = pd.merge(train_df, building_info_df, on='건물번호', how='left')
cluster_df['일시_dt'] = pd.to_datetime(cluster_df['일시'], format='%Y%m%d %H')
cluster_df['hour'] = cluster_df['일시_dt'].dt.hour
cluster_df['weekday'] = cluster_df['일시_dt'].dt.weekday

hourly_profile = cluster_df.groupby(['건물유형', 'hour'])['전력소비량(kWh)'].mean().unstack().fillna(0)
weekday_profile = cluster_df.groupby(['건물유형', 'weekday'])['전력소비량(kWh)'].mean().unstack().fillna(0)
building_type_profiles = pd.concat([hourly_profile, weekday_profile], axis=1)

scaler = StandardScaler()
scaled_profiles = scaler.fit_transform(building_type_profiles)
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
building_type_profiles['cluster'] = kmeans.fit_predict(scaled_profiles)
cluster_map = building_type_profiles[['cluster']].reset_index()
building_info_df = pd.merge(building_info_df, cluster_map, on='건물유형', how='left')


Feature Enginnering

In [24]:
# 이상치 처리
train_df['일시_dt_temp'] = pd.to_datetime(train_df['일시'], format='%Y%m%d %H')
train_df['hour_temp'] = train_df['일시_dt_temp'].dt.hour
train_df['weekday_temp'] = train_df['일시_dt_temp'].dt.weekday
for building_num in train_df['건물번호'].unique():
    building_mask = train_df['건물번호'] == building_num
    power_series = train_df.loc[building_mask, '전력소비량(kWh)']
    Q1, Q3 = power_series.quantile(0.25), power_series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound, upper_bound = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    outlier_indices = power_series[(power_series < lower_bound) | (power_series > upper_bound)].index
    if not outlier_indices.empty:
        non_outlier_mask = building_mask & ~train_df.index.isin(outlier_indices)
        replacement_map = train_df.loc[non_outlier_mask].groupby(['weekday_temp', 'hour_temp'])['전력소비량(kWh)'].mean()
        for idx in outlier_indices:
            weekday, hour = train_df.loc[idx, 'weekday_temp'], train_df.loc[idx, 'hour_temp']
            replacement_value = replacement_map.get((weekday, hour), power_series.median())
            train_df.loc[idx, '전력소비량(kWh)'] = replacement_value
train_df = train_df.drop(columns=['일시_dt_temp', 'hour_temp', 'weekday_temp'])

# Combine train and test for consistent feature creation
test_df['전력소비량(kWh)'] = np.nan
all_df = pd.concat([train_df, test_df], ignore_index=True)

# 변수 제거 및 추가
all_df = all_df.drop(columns=['일조(hr)', '일사(MJ/m2)'])
all_df = pd.merge(all_df, building_info_df.drop(columns=['건물유형']), on='건물번호', how='left')

# 파생변수 생성
holidays = ['2024-06-06', '2024-08-15'] # test 기간에 해당하는 공휴일만 사용
all_df['일시_dt'] = pd.to_datetime(all_df['일시'], format='%Y%m%d %H')
all_df['주말여부'] = all_df['일시_dt'].dt.weekday.isin([5, 6]).astype(int)
all_df['is_holiday'] = all_df['일시_dt'].dt.date.astype(str).isin(holidays).astype(int)
best_bp = 24
all_df['냉방도일'] = (all_df['기온(°C)'] > best_bp) * (all_df['기온(°C)'] - best_bp)
all_df['hour'] = all_df['일시_dt'].dt.hour
all_df['weekday'] = all_df['일시_dt'].dt.weekday
all_df['month'] = all_df['일시_dt'].dt.month
all_df['day'] = all_df['일시_dt'].dt.day

# 주기성: 삼각함수, Lag, 이동평균
all_df['hour_sin'] = np.sin(2 * np.pi * all_df['hour'] / 24)
all_df['hour_cos'] = np.cos(2 * np.pi * all_df['hour'] / 24)
all_df['weekday_sin'] = np.sin(2 * np.pi * all_df['weekday'] / 7)
all_df['weekday_cos'] = np.cos(2 * np.pi * all_df['weekday'] / 7)
all_df = all_df.sort_values(by=['건물번호', '일시'])
# [수정] 단기 Lag 변수인 lag_1h를 제외합니다.
# all_df['lag_1h'] = all_df.groupby('건물번호')['전력소비량(kWh)'].shift(1)
all_df['MA_24h'] = all_df.groupby('건물번호')['전력소비량(kWh)'].shift(1).rolling(window=24).mean()
all_df['MA_168h'] = all_df.groupby('건물번호')['전력소비량(kWh)'].shift(1).rolling(window=168).mean()

# K-Means 클러스터 라벨을 변수로 사용
all_df = pd.get_dummies(all_df, columns=['cluster'], prefix='cluster')

모델 학습 및 예측

In [25]:
all_df = all_df.drop(columns=['일시_dt', 'hour', 'weekday'])
train_processed_df = all_df[all_df['전력소비량(kWh)'].notna()].copy()
test_processed_df = all_df[all_df['전력소비량(kWh)'].isna()].drop(columns=['전력소비량(kWh)'])

# 결측치 처리
train_processed_df.fillna(method='bfill', inplace=True)
train_processed_df.fillna(method='ffill', inplace=True)
test_processed_df.fillna(method='bfill', inplace=True)
test_processed_df.fillna(method='ffill', inplace=True)

train_final_df = train_processed_df
test_final_df = test_processed_df
for df in [train_final_df, test_final_df]:
    for col in ['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']:
        if col in df.columns:
            df[col] = df[col].replace('-', 0).astype(float).fillna(0)

target = '전력소비량(kWh)'


train_final_df[target] = np.log1p(train_final_df[target])

train_cols = [col for col in train_final_df.columns if col != target]
test_final_df = test_final_df.reindex(columns=train_cols, fill_value=0)
initial_features = [col for col in train_cols if col not in ['num_date_time', '일시']]

X_fs = train_final_df[initial_features]
y_fs = train_final_df[target]

from sklearn.model_selection import train_test_split
X_train_fs, X_val_fs, y_train_fs, y_val_fs = train_test_split(X_fs, y_fs, test_size=0.2, random_state=42)

perm_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42, n_jobs=-1)
perm_model.fit(X_train_fs, y_train_fs)

result = permutation_importance(perm_model, X_val_fs, y_val_fs, n_repeats=5, random_state=42, n_jobs=-1)
perm_importances = pd.DataFrame({'feature': initial_features, 'importance_mean': result.importances_mean})
perm_importances = perm_importances.sort_values(by='importance_mean', ascending=False)
ranked_features = perm_importances[perm_importances['importance_mean'] > 1e-5]['feature'].tolist()
selected_features = ranked_features
print(f"\n  >> 총 {len(initial_features)}개 변수 중 {len(ranked_features)}개의 유효 변수를 선택했습니다.")
print("------------------------------------")

evaluation_results = []
all_predictions = []
print("\n각 건물별 모델 학습, 평가 및 예측을 시작합니다...")

for building_num in range(1, 101):
    train_building = train_final_df[train_final_df['건물번호'] == building_num]
    test_building = test_final_df[test_final_df['건물번호'] == building_num]

    X = train_building[selected_features]
    y = train_building[target]
    X_test = test_building[selected_features]


    split_point = int(len(X) * 0.8)
    X_train_split, X_val = X.iloc[:split_point], X.iloc[split_point:]
    y_train_split, y_val = y.iloc[:split_point], y.iloc[split_point:]

    eval_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.05, max_depth=5, subsample=0.8, colsample_bytree=0.8, random_state=42, n_jobs=-1, early_stopping_rounds=50)
    eval_model.fit(X_train_split, y_train_split, eval_set=[(X_val, y_val)], verbose=False)

    y_pred_log = eval_model.predict(X_val)


    y_val_original = np.expm1(y_val)
    y_pred_original = np.expm1(y_pred_log)
    y_pred_original[y_pred_original < 0] = 0 # 전력량은 음수가 될 수 없으므로 0으로 처리

    rmse = np.sqrt(mean_squared_error(y_val_original, y_pred_original))
    mae = mean_absolute_error(y_val_original, y_pred_original)
    r2 = r2_score(y_val_original, y_pred_original)
    smape_score = smape(y_val_original, y_pred_original)
    evaluation_results.append({'건물번호': building_num, 'RMSE': rmse, 'MAE': mae, 'R2': r2, 'SMAPE': smape_score})


    final_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.05, max_depth=5, subsample=0.8, colsample_bytree=0.8, random_state=42, n_jobs=-1)
    final_model.fit(X, y)

    final_preds_log = final_model.predict(X_test)

    final_preds = np.expm1(final_preds_log)
    final_preds[final_preds < 0] = 0 # 음수 예측값 0으로 처리

    pred_df = pd.DataFrame({'num_date_time': test_building['num_date_time'].values, 'answer': final_preds})
    all_predictions.append(pred_df)

    if building_num % 10 == 0:
        print(f"  ... {building_num}번 건물 처리 완료.")

results_df = pd.DataFrame(evaluation_results)
average_performance = results_df.drop(columns='건물번호').mean()
print("\n--- 모델 성능 평가 결과 (시간 기반 검증 적용) ---")
print(f"Average RMSE (평균 오차): {average_performance['RMSE']:.3f} kWh")
print(f"Average MAE (평균 절대 오차): {average_performance['MAE']:.3f} kWh")
print(f"Average R2 Score (모델 설명력): {average_performance['R2']:.3f}")
print(f"Average SMAPE (실제 예상 점수): {average_performance['SMAPE']:.3f} %")

submission_df = pd.concat(all_predictions, ignore_index=True)
sample_submission_df = pd.read_csv('sample_submission.csv')
submission_df = submission_df.set_index('num_date_time').reindex(sample_submission_df['num_date_time']).reset_index()

submission_df.to_csv('submission_v2.csv', index=False, encoding='utf-8')
print("'submission_v2.csv' 파일 생성이 완료되었습니다.")



/tmp/ipython-input-2539027727.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_processed_df.fillna(method='bfill', inplace=True)
/tmp/ipython-input-2539027727.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_processed_df.fillna(method='ffill', inplace=True)
/tmp/ipython-input-2539027727.py:8: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_processed_df.fillna(method='bfill', inplace=True)
/tmp/ipython-input-2539027727.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_processed_df.fillna(method='ffill', inplace=True)



  >> 총 24개 변수 중 22개의 유효 변수를 선택했습니다.
------------------------------------

각 건물별 모델 학습, 평가 및 예측을 시작합니다...
  ... 10번 건물 처리 완료.
  ... 20번 건물 처리 완료.
  ... 30번 건물 처리 완료.
  ... 40번 건물 처리 완료.
  ... 50번 건물 처리 완료.
  ... 60번 건물 처리 완료.
  ... 70번 건물 처리 완료.
  ... 80번 건물 처리 완료.
  ... 90번 건물 처리 완료.
  ... 100번 건물 처리 완료.

--- 모델 성능 평가 결과 (시간 기반 검증 적용) ---
Average RMSE (평균 오차): 238.631 kWh
Average MAE (평균 절대 오차): 168.457 kWh
Average R2 Score (모델 설명력): 0.470
Average SMAPE (실제 예상 점수): 6.251 %
'submission_v2.csv' 파일 생성이 완료되었습니다.
